<center>
<hr>
<h1>INF-477. Redes Neuronales Artificiales.</h1>
<h2>Tarea 2 - Autoencoders, RBMs y ConvNets</h2>
<hr>
</center>

<div style="width:25%; display: inline-block"></div>
<div style="width:25%; display: inline-block">
    <b>Juan Carlos Garcés Bernt</b><br>
    jcgarces@alumnos.inf.utfsm.cl
</div>
<div style="width:25%; display: inline-block;">
    <b>Natalia Gonzales</b><br>
    a@a.cl
</div>
<div style="width:25%; display: inline-block"></div>


<h1>1 Entrenamiento de Autoencoders (AEs) y RBMs en MNIST</h1>

Como hemos discutido en clases, las RBM's y posteriormente los AE's, fueron un componente crucial en el
desarrollo de los modelos que entre 2006 y 2010 vigorizaron el área de las redes neuronales artificiales con
logros notables de desempeño en diferentes tareas de aprendizaje automático.<br>
En esta sección aprenderemos a utilizar estos modelos en tres escenarios clásicos: reducción de dimensionalidad,
denoising y pre-entrenamiento. Con este objetivo en mente, utilizaremos un dataset denominado
*MNIST*, bastante conocido en el área e introducido por Yann LeCunn hacia 1998 en un trabajo que,
junto al Neocognitron de Fukushima, se considera uno de los principales antecedentes de las redes convolucionales
modernas. Se trata de una colección de $70.000$ imágenes de $28 \times 28$ pixeles correspondientes a
dígitos manuscritos (números entre $0$ y $9$). En su versión tradicional, la colección se encuentra separada en
dos subconjuntos: uno de entrenamiento de $60.000$ imágenes y otro de test de $10.000$ imágenes. La tarea
consiste en entrenar un programa para que aprenda a identificar correctamente el dígito representado en la
imagen.
<img src="img/Fig1.png">
Fig. 1: Dataset MNIST y visualización obtenida usando las primeras dos componentes principales.

a) Escriba una función que cargue los datos desde el repositorio de keras, normalice las imágenes de
modo que los pixeles queden en $[0, 1]$, transforme las imágenes en vectores $(\in \mathbb{R}^{784})$ y devuelva tres
subconjuntos disjuntos: uno de entrenamiento, uno de validación y uno de pruebas. La normalización
permite interpretar cada valor como una probabilidad de "activación" del un pixel. El conjunto de
pruebas será aquel por defecto. Para la construcción del subconjunto de validación su función recibirá
un parámetro **NVAL**, cuyo valor por defecto será $1000$. El conjunto de validación se construirá utilizando
los últimos **NVAL** casos del conjunto del entrenamiento por defecto. El conjunto de entrenamiento
consistirá en las primeras $60000$ - **NVAL** imágenes.

In [1]:
from keras.datasets import mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
x_val = x_train[-nval:]
y_val = y_train[-nval:]
x_train = x_train[:-nval]
y_train = y_train[:-nval]
Y_train = np_utils.to_categorical(y_train, 10)
Y_val = np_utils.to_categorical(y_val, 10)
Y_test = np_utils.to_categorical(y_test, 10)

Using Theano backend.


15261696/15296311 [============================>.] - ETA: 0s

NameError: name 'nval' is not defined

<h2>1.1 Reducción de Dimensionalidad</h2>
Construir una representación de menor dimensionalidad de un objeto en $\mathbb{R}^d$, consiste en construir una función $\phi: \mathbb{R}^d \rightarrow \mathbb{R}^{d'},$ con $d' \ll d$ que preserve lo mejor posible la "información" original. Obtener tal representación es útil desde un punto de vista computacional (compresión) y estadístico (permite construir modelos con un menor número de parámetros libres). Una técnica de reducción de dimensionalidad se denomina *no super-visada* cuando no hace uso de información acerca de las clases a las que pertenecen los datos de entrenamiento, marco de trabajo útil cuando dicha información no está disponible. Un AE y una RBM se pueden considerar métodos no-supervisados de reducción de dimensionalidad.

a) Entrene un AR básico (1capa escondida) para generar una representación de MNIST en $d' = 2,8,32,64$ dimensiones. Determine el porcentaje de compresión obtenido y el error de recontrucción en cada caso.
¿Mejora el resultado si elegimos una función de activación ReLU para el Encoder? ¿Podría utilizarse una ReLU en el decoder?

In [2]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD

input_img = Input(shape=(784,))
encoded = Dense(32, activation='sigmoid')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input=input_img, output=decoded)
encoder = Model(input=input_img, output=encoded)
encoded_input = Input(shape=(32,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))
autoencoder.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
autoencoder.fit(x_train,x_train,nb_epoch=50,batch_size=25,shuffle=True,validation_data=(x_val, x_val))
autoencoder.save('basic_autoencoder_768x32.h5')
#save other stuff ...

NameError: name 'x_val' is not defined

b) Para verificar la calidad del modelo obtenido, compare visualmente la recontruscción que logra hacer el autoencoder desde la representación en $\mathbb{R}^{d'}$ para algunas imágenes del conjunto de pruebas. Determine si la percepción visual se correpsonde con el error de recontrucción observado. Comente.

In [3]:
from keras.models import load_model
autoencoder = load_model('basic_autoencoder_768x32.h5')
#load other stuff ...
encoded_test = encoder.predict(x_test)
decoded_test = decoder.predict(encoded_test)
import matplotlib
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

OSError: Unable to open file (Unable to open file: name = 'basic_autoencoder_768x32.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

c) Para verificar la calidad de la representación obtenida, implemente el siguiente clasificador, denominado *kNN* (k-nearest neighbor): dada una imagen $x$, el clasificador busca las $k=10$ imágenes de entrenamiento más similares $N_{x} = \{ x^{(k_i)}\}_{i=1}^{10}$ (de acuerdo a una distancia, e.g. euclidiana) y predice como clase, la etiqueta más popular entre las imágenes $N_x$. Mida el error de pruebas obtenido construyendo este clasificador sobre la data original y luego sobre la data reducida. Compare además los tiempos medios de predicción en ambos escenarios.

In [4]:
encoded_train = encoder.predict(x_train)
encoded_test = encoder.predict(x_test)
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(10)
clf.fit(encoded_train, y_train)
clf.fit(encoded_train, y_train)
score = clf.score(encoded_test,y_test)
print('Classification Accuracy %.2f' % score)

Classification Accuracy 0.91


d) Para verificar la calidad de la representación obtenida, implemente *k-means* (un método básico de
agrupamiento) sobre la representación obtenida por el autoencoder. Mida la calidad del agrupamiento
obtenido sobre los datos reducidos utilizando la métrica denominada ARI (*Adjusted Rand Index*) y
la función de desempeño (que llamaremos *clustering accuracy*) definida en el código de ejemplo que se
proporciona más abajo. Compare el resultado con el agrupamiento obtenido sobre los datos originales.

In [5]:
def clustering_accuracy(pred_labels,y,nclusters=10):
    true_pred = 0.0
    for i in range(0,nclusters):
        mvlabel = np.argmax(np.bincount(y[pred_labels==i]))
        true_pred += sum(y[pred_labels==i] == mvlabel)
    return true_pred/len(y)

from sklearn.cluster import KMeans
from sklearn import metrics
model = KMeans(n_clusters=10)
labels_pred = model.fit_predict(encoded_train)
score = metrics.adjusted_rand_score(y_train, labels_pred)
print ('Clustering ARI %.2f' % score)
print ('Clustering ACC %.2f' % clustering_accuracy(labels_pred,y_train))

Clustering ARI 0.27
Clustering ACC 0.51


e) $\star$ Compare la calidad de la representación reducida obtenida por el autoencoder básico con aquella
obtenida vía PCA utilizando el mismo número de dimensiones $d'$. Considere los $4$ criterios que hemos
utilizado hasta el momento, i.e., error de reconstrucción, visualización de la reconstrucción, desempeño
en clasificación (vía kNN) y desempeño en agrupamiento (vía kMeans). Comente.

In [6]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
pca = PCA(n_components=32)
pca.fit(x_train)
pca_train = pca.transform(x_train)
pca_test = pca.transform(x_test)
clf = KNeighborsClassifier(10)
clf.fit(pca_train, y_train)
score = clf.score(pca_test,y_test)
print('PCA SCORE %.2f' % score)

PCA SCORE 0.97


f) Entrene una RBM binaria básica para generar una representación de MNIST en $d' = 2, 8, 32, 64$ dimensiones.
Determine el porcentaje de compresión obtenido y el error de reconstrucción en cada caso.
Compare con los resultados obtenidos por el autoencoder utilizando los $3$ criterios que hemos utilizado
hasta el momento, i.e., error de reconstrucción, desempeño en clasificación (vía kNN) y desempeño en
agrupamiento (vía kMeans).

In [7]:
from sklearn.neural_network import BernoulliRBM
import numpy as np
import pickle ##to save trained models
model = BernoulliRBM(n_components=32, batch_size=25,learning_rate=0.05,verbose=1, n_iter=50) ##n_components is d
model.fit(x_train)##Train using persistent Gibbs chains
fileo = open('basicRBM.pickle','wb')
pickle.dump(model,fileo)
fileo.close()

[BernoulliRBM] Iteration 1, pseudo-likelihood = -164.30, time = 3.63s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -138.89, time = 4.64s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -130.79, time = 4.18s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -126.38, time = 4.32s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -121.56, time = 4.36s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -119.62, time = 4.22s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -119.43, time = 4.34s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -118.32, time = 4.04s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -119.64, time = 4.88s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -117.82, time = 4.40s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -116.32, time = 4.35s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -116.92, time = 4.19s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -116.54, time = 4.19s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -116.76, time = 4.52s
[BernoulliRBM] 

g) $\star \star$ Modifique el autoencoder básico construido en (a) para implementar un *deep autoencoder* (deep
AE), es decir, un autoencoder con al menos dos capas ocultas. Demuestre experimentalmente que este
autoencoder puede mejorar significativamente la compresión obtenida por PCA utilizando el mismo
número de dimensiones $d'$. Experimente con $d' = 2, 4, 8, 16, 32$ y distintas profundidades ($L = 2, 3, 4$).
Considere en esta comparación los 3 criterios que hemos utilizado hasta el momento, i.e., error de
reconstrucción, desempeño en clasificación (vía kNN) y desempeño en agrupamiento (vía kMeans).
Comente.

In [9]:
target_dim = 2 #try other and do a nice plot
input_img = Input(shape=(784,))
encoded1 = Dense(1000, activation='relu')(input_img)
encoded2 = Dense(500, activation='relu')(encoded1)
encoded3 = Dense(250, activation='relu')(encoded2)
encoded4 = Dense(target_dim, activation='relu')(encoded3)
decoded4 = Dense(250, activation='relu')(encoded4)
decoded3 = Dense(500, activation='relu')(encoded3)
decoded2 = Dense(1000, activation='relu')(decoded3)
decoded1 = Dense(784, activation='sigmoid')(decoded2)
autoencoder = Model(input=input_img, output=decoded1)
encoder = Model(input=input_img, output=encoded3)
autoencoder.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
autoencoder.fit(x_train,x_train,nb_epoch=50,batch_size=25,shuffle=True,validation_data=(x_val, x_val))
autoencoder.save('my_autoencoder_768x1000x500x250x2.h5')
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
pca = PCA(n_components=target_dim)
pca.fit(x_train)

NameError: name 'x_val' is not defined

h) Para el caso $d'=2$ de los experimentos anteriores, genere un gráfico que muestre la representación aprendida. Con este fin, utilice por ejemplo la herramienta de visualización TSNE disponible en *sklearn*. Compare cualitativamente el resultado con aquel obtenido usando PCA.

In [10]:
nplot=5000 #warning: mind your memory!
encoded_train = encoder.predict(x_train[:nplot])
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=0)
encoded_train = model.fit_transform(encoded_train)

colors={0:'b',1:'g',2:'r',3:'c',4:'m',5:'y',6:'k',7:'orange',8:'darkgreen',9:'maroon'}
markers={0:'o',1:'+',2:'v',3:'<',4:'>',5:'^',6:'s',7:'p',8:'*',9:'x'}
plt.figure(figsize=(10, 10))
for idx in xrange(0,nplot):
    label = y_train[idx]
    line = plt.plot(encoded_train[idx][0], encoded_train[idx][1],color=colors[label], marker=markers[label], markersize=6)
pca_train = pca.transform(x_train)
encoded_train = pca_train[:nplot]
#... plot PCA

NameError: name 'plt' is not defined

i) Contruya una función que permita visualizar algunos de los pesos aprendidos por las neuronas de la primera capa del autoencoder. Muestre el resultado para las mejores redes conseguidas en los ítem anteriores.

j) Estudie como cambian los resultados del modelo construido en (a) si se impone simetría, es decir, si se trabaja con *tied weights*.

<h2>1.2 Denoising </h2>
Como hemos discutido en clases, un *denoising autoencoder* (dAE) es esencialmente un *autoencoder* entrenado
para reconstruir ejemplos parcialmente corruptos. Varios autores han demostrado que mediante esta modificación simple es posible obtener representaciones latentes más robustas y significativas que aquellas obtenidas por un AE básico. En esta sección exploraremos la aplicación más "natural" o "directa" del método.

a) Genere artificialmente una versión corrupta de las imágenes en MNIST utilizando el siguiente modelo de ruido (masking noise): si $x \in \mathbb{R}^d$ es una de las imágenes originales, la versión ruidosa $\tilde x$ se obtiene como $\tilde x = x \odot \xi$ donde $\odot $ denota el producto de Hadamard ( componente a componente) y $\xi \in \mathbb{R}^d$ es un vector aleatorio binario con componentes Ber($p$) independientes.

In [11]:
from numpy.random import binomial
noise_level = 0.1
noise_mask = binomial(n=1,p=noise_level,size=x_train.shape)
noisy_x_train = x_train*noise_mask
noise_mask = binomial(n=1,p=noise_level,size=x_val.shape)
noisy_x_val = x_val*noise_mask
noise_mask = binomial(n=1,p=noise_level,size=x_test.shape)
noisy_x_test = x_test*noise_mask

NameError: name 'x_val' is not defined